In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='itp-pilot', subscription_id='4aaa645c-5ae2-4ae9-a17a-84b9023bc56a', resource_group='itp-pilot-ResGrp')
#ws = Workspace.get(name='cmaksphillywestcentralus', subscription_id='06237282-1ddd-4677-9878-5bc19cc0001a', resource_group='ITPTestingWorkers')
#ws = Workspace.get(name='cmk8s-test-ws', subscription_id='4aaa645c-5ae2-4ae9-a17a-84b9023bc56a', resource_group='cmk8s-test-rg')
#ws = Workspace.get(name='resrchvc', subscription_id='46da6261-2167-4e71-8b0d-f4a45215ce61', resource_group='researchvc')

train_module_func = Module.from_yaml(ws, './train-score-eval/mpi_train.yaml')
score_module_func = Module.from_yaml(ws, './train-score-eval/score.yaml')
eval_module_func = Module.from_yaml(ws, './train-score-eval/eval.yaml')
compare_module_func = Module.from_yaml(ws, './train-score-eval/compare2.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [2]:
# define a sub pipeline
@dsl.pipeline(name = 'Train-Score-Eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, learning_rate):
    train = train_module_func(
        training_data=input_data, 
        max_epochs=5, 
        learning_rate=learning_rate,
        run_for_minutes=10)
    train.runsettings.configure(node_count=1, process_count_per_node=1)
    train.k8srunsettings.priorityconfiguration.is_preemptible=True
    train.k8srunsettings.priorityconfiguration.priority=200
    #train.inputs.training_data.configure(mode='download',)
   
    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)

    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [3]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'itp-system-test-pipeline-run', 
              description = 'select best model trained with different learning rate',
#              default_compute_target = 'aks-gpu')
              default_compute_target = ('k80-16-a', 'Cmk8s')
#               default_compute_target = ('itpseasiav100cl', 'Cmk8s'))
#               default_compute_target = 'gpu-cluster')
)
def dummy_automl_pipeline():
    train_and_evalute_model1 = training_pipeline(train_data, 0.01)
    train_and_evalute_model2 = training_pipeline(train_data, 0.02)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {**compare.outputs}

# create a pipeline
pipeline = dummy_automl_pipeline()

In [4]:
#pipeline.validate()

In [5]:
pipeline.submit(experiment_name='samples').wait_for_completion()

Submitted PipelineRun 5b074a33-d6cb-41d1-9fc0-913ed7644f5d
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/5b074a33-d6cb-41d1-9fc0-913ed7644f5d?wsid=/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourcegroups/itp-pilot-ResGrp/workspaces/itp-pilot
PipelineRunId: 5b074a33-d6cb-41d1-9fc0-913ed7644f5d
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/5b074a33-d6cb-41d1-9fc0-913ed7644f5d?wsid=/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourcegroups/itp-pilot-ResGrp/workspaces/itp-pilot


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_cf4e63c4-5280-4bb2-bc6f-ba5dd685630a_widget', env_json='{}', graph_jso…

<RunStatus.failed: 'Failed'>